# DBLP Anomalies

Before conducting the following analysis, we preprocessed the DBLP data set to compute the number of articles and coauthors by year, and the cummulative sum of distinct coauthors up to each year. Besides, we compute the number of articles the authors published on each venue by year, as well as the author-article graph. We also masked the authors' names.

## Definitions

In [1]:
library(stringr)
library(data.table)
library(ggplot2)
library(gridExtra)
library(rjson)
library(dtwclust)
library(factoextra)
library(cluster)
library(abodOutlier)
library(corrplot)
library(igraph)
library(birankr)
library(IRdisplay)
library(xtable)
library(matrixStats)
library(dplyr)
library(data.table)

Loading required package: proxy


Attaching package: ‘proxy’


The following objects are masked from ‘package:stats’:

    as.dist, dist


The following object is masked from ‘package:base’:

    as.matrix


Loading required package: dtw

Loaded dtw v1.22-3. See ?dtw for help, citation("dtw") for use in publication.


dtwclust:
Setting random number generator to L'Ecuyer-CMRG (see RNGkind()).
To read the included vignettes type: browseVignettes("dtwclust").
See news(package = "dtwclust") after package updates.

Welcome! Want to learn more? See two factoextra-related books at https://goo.gl/ve3WBa

corrplot 0.84 loaded


Attaching package: ‘igraph’


The following objects are masked from ‘package:stats’:

    decompose, spectrum


The following object is masked from ‘package:base’:

    union


Loading required package: Matrix


Attaching package: ‘xtable’


The following object is masked from ‘package:IRdisplay’:

    display



Attaching package: ‘dplyr’


The following object is mask

In [2]:
# for STEP in {1..23}; do 
# awk -F";" '{print $1";"$2";"$3";"$7";"$8}' tip_decomposition-model-4-step-$STEP-partition-1 > /tmp/file.tmp && mv /tmp/file.tmp tip_decomposition-model-4-step-$STEP-partition-1;
# done

In [3]:
options(scipen = 999)

In [4]:
read.author.index <- function(f.name) {
  df <- read.csv2(f.name, header = T, sep = ";", stringsAsFactors = F)
  rownames(df) <- as.character(df$id)
  df$author.first.name <- sapply(str_split(df$author, " "), function(a) {a[1]})
  df$author.first.name <- paste(df$author.first.name)#, " (", df$id ,")", sep = "")
  df
}

read.nodes <- function(f.name) {
  df <- read.csv2(f.name, header = T, sep = ";", stringsAsFactors = F)
  df$Time <- df$Time - 1
  df$Degree <- as.numeric(df$Degree)
  df
}

read.graph.author.venue.one.step <- function(f.name) {
    df <- read.csv2(f.name, header = T, sep = ";", stringsAsFactors = F)
    df <- df %>% 
        group_by(author, venue) %>% 
        summarise(weight = n())
    df
}

VENUE.SHIFT <- 10000000

read.graph.author.venue.all.steps.by.year <- function(dir.name, window.begin, window.end) {
    window.begin <- window.begin - 2000 + 1
    window.end <- window.end - 2000 + 1
    all.steps <- list.files(
        dir.name, 
        pattern = "graph-\\d+$"
    )
    
    # Sort by step  
    tmp.sort <- str_match(all.steps, "graph-(\\d+)")
    all.steps <- all.steps[order(as.integer(tmp.sort[, 2]))]
    all.steps <- all.steps[window.begin:window.end]
    
    all.files <- paste(dir.name, "/", all.steps, sep = "")
    all.graphs <- lapply(all.files, function(f.name) {
        tmp <- read.graph.author.venue.one.step(f.name)
        tmp$time <- as.integer(str_match(f.name, "graph-(\\d+)")[, 2])
        tmp
    })
    all.graphs <- do.call(rbind, all.graphs)
    
    # Shift venue ids
    all.graphs$venue <- all.graphs$venue + VENUE.SHIFT
    
    all.graphs %>%
        group_by(author, venue, time) %>%
        summarise (weight = sum(weight))
}

read.graph.author.venue.all.steps <- function(dir.name, window.begin, window.end) {
    window.begin <- window.begin - 2000 + 1
    window.end <- window.end - 2000 + 1
    all.steps <- list.files(
        dir.name, 
        pattern = "graph-\\d+$"
    )
    
    # Sort by step  
    tmp.sort <- str_match(all.steps, "graph-(\\d+)")
    all.steps <- all.steps[order(as.integer(tmp.sort[, 2]))]
    all.steps <- all.steps[window.begin:window.end]
    
    
    all.files <- paste(dir.name, "/", all.steps, sep = "")
    all.graphs <- lapply(all.files, function(f.name) {
        read.graph.author.venue.one.step(f.name)
    })
    all.graphs <- do.call(rbind, all.graphs)
    
    # Shift venue ids
    all.graphs$venue <- all.graphs$venue + VENUE.SHIFT
    
    all.graphs %>%
        group_by(author, venue) %>%
        summarise (weight = sum(weight))
}


read.graph.author.publication.one.step <- function(f.name) {
    df <- read.csv2(f.name, header = T, sep = ";", stringsAsFactors = F)
    df
}


read.graph.author.publication.all.steps <- function(dir.name, window.begin, window.end) {
    window.begin <- window.begin - 2000 + 1
    window.end <- window.end - 2000 + 1
    all.steps <- list.files(
        dir.name, 
        pattern = "graph-\\d+$"
    )
    
    # Sort by step  
    tmp.sort <- str_match(all.steps, "graph-(\\d+)")
    all.steps <- all.steps[order(as.integer(tmp.sort[, 2]))]
    all.steps <- all.steps[window.begin:window.end]
    
    all.files <- paste(dir.name, "/", all.steps, sep = "")
    all.graphs <- lapply(all.files, function(f.name) {
        read.graph.author.publication.one.step(f.name)
    })
    all.graphs <- do.call(rbind, all.graphs)
    
    all.graphs
}

read.graph.author.venue.all.steps.weighted.incidence <- function(dir.name, window.begin, window.end) {
    window.begin <- window.begin - 2000 + 1
    window.end <- window.end - 2000 + 1
    all.steps <- list.files(
        dir.name, 
        pattern = "graph-\\d+$"
    )
    
    # Sort by step  
    tmp.sort <- str_match(all.steps, "graph-(\\d+)")
    all.steps <- all.steps[order(as.integer(tmp.sort[, 2]))]
    all.steps <- all.steps[window.begin:window.end]
    
    all.files <- paste(dir.name, "/", all.steps, sep = "")
    all.graphs <- lapply(all.files, function(f.name) {
        read.graph.author.venue.one.step(f.name)
    })
    all.graphs <- do.call(rbind, all.graphs)
    
    # Shift venue ids
    all.graphs$venue <- all.graphs$venue + VENUE.SHIFT
    
    # Weight is the summation of the incident edges.
    # Reincident edges are increasewd by one each time.
    # Computed as an aritmetic sequence increased by 1.
    all.graphs %>%
        group_by(author, venue) %>%
        summarise (
            weight = n()*(1 + sum(weight)) / 2
        )
}

read.graph.author.venue.all.steps.weighted.incidence.normalized.by.venue <- function(dir.name, window.begin, window.end) {
    window.begin <- window.begin - 2000 + 1
    window.end <- window.end - 2000 + 1
    all.steps <- list.files(
        dir.name, 
        pattern = "graph-\\d+$"
    )
    
    # Sort by step  
    tmp.sort <- str_match(all.steps, "graph-(\\d+)")
    all.steps <- all.steps[order(as.integer(tmp.sort[, 2]))]
    all.steps <- all.steps[window.begin:window.end]

    
    
    all.files <- paste(dir.name, "/", all.steps, sep = "")
    all.graphs <- lapply(all.files, function(f.name) {
        read.graph.author.venue.one.step(f.name)
    })
    all.graphs <- do.call(rbind, all.graphs)
    
    # Shift venue ids
    all.graphs$venue <- all.graphs$venue + VENUE.SHIFT
    
    # Weight is proportion of papers publised by the author in each venue
    all.graphs <- all.graphs %>%
        group_by(author, venue) %>%
        summarise (
            weight = n()
        )
    
    all.graphs %>%
        group_by(venue) %>%
        mutate (
            weight = weight / sum(weight)
        )
}

read.venue.index <- function(venue.index.file) {
    df <- read.csv2(venue.index.file, header = T, sep = ";", stringsAsFactors = F)
    df$id <- df$id + VENUE.SHIFT
    df
}

read.graph.author.venue.as.matrix <- function(dir.name) {
    weighted.edge.list <- read.graph.author.venue.all.steps(dir.name)
    birankr::sparsematrix_from_edgelist(
      data = weighted.edge.list,
      sender_name = "author",
      receiver_name = "venue",
      weight_name = "weight",
      is_bipartite = T
    )

}

read.coauthors.cumsum.over.time <- function(processed.dir) {
  cumsum.file <- paste0(processed.dir, "/coauthors-cumulative-sum.csv", sep="")
  coauthors.by.year.cumsum <- read.csv(cumsum.file, header = T, sep=";")
  coauthors.by.year.cumsum$coauthor.cumsum <- coauthors.by.year.cumsum$cumsum
  coauthors.by.year.cumsum$cumsum <- NULL
  coauthors.by.year.cumsum
}

read.all.tip.files <- function(tipness.files.names, filter.nodes=F, anoms=NULL, control=NULL) {
  all.t <- NULL
  for (i in 1:length(tipness.files.names)) {
    dir.name.t <- tipness.files.names[i]
    
    nodes.t <- read.nodes(dir.name.t)
    
    if (filter.nodes) {
      nodes.t <- nodes.t %>% filter(Node %in% c(anoms, control))
    }
    
    if (is.null(all.t)) {
      all.t <- nodes.t
    } else {
      all.t <- rbind(all.t, nodes.t)
    }
  }
  all.t$Node <- as.factor(all.t$Node)
  all.t
}

read.coauthors.cumsum.over.time <- function(processed.dir) {
  cumsum.file <- paste0(processed.dir, "/coauthors-cumulative-sum.csv", sep="")
  coauthors.by.year.cumsum <- read.csv(cumsum.file, header = T, sep=";")
  coauthors.by.year.cumsum$coauthor.cumsum <- coauthors.by.year.cumsum$cumsum
  coauthors.by.year.cumsum$cumsum <- NULL
  coauthors.by.year.cumsum
}

In [5]:
compute.academic.birth <- function(all.t) {
    all.t %>% 
        group_by(Node) %>%
        summarize(
            academic.birth = min(Time)
        )
}

In [6]:
compute.derivatives.in.time <- function(all.t, min.publications) {

    window.lag.computation <- function(value.vector, multiple) {
    
        ifelse(
                seq_along(value.vector) == multiple,
                value.vector[multiple],
                ifelse(
                    seq_along(value.vector) %% multiple == 0, 
                    value.vector - lag(value.vector, multiple),
                    0
                )
        )
    }
    
    all.t <- all.t %>%
            tidyr::complete(Node,Time) %>%
            group_by(Node) %>% 
            arrange(Node, Time) %>% 
            mutate(
              all.papers = sum(Degree.model.4, na.rm = T),
              cumulative.papers = cumsum(ifelse(is.na(Degree.model.4), 0, Degree.model.4)), 
              cumulative.coauthors = max(coauthor.cumsum, na.rm=T)
            ) %>%
            filter(cumulative.papers > 0) %>% # Remove extra lines before authors' start publishing
            mutate(
              first.time.all.papers = (Time[cumulative.papers == all.papers])[1],
                
              diff.papers = Degree.model.4,
              diff.cumulative.coauthors = coauthor.cumsum - lag(coauthor.cumsum),
              diff.coauthors = Neighbors.model.4,

              diff.papers.2.years = window.lag.computation(cumulative.papers, 2),
              diff.cumulative.coauthors.2.years = window.lag.computation(coauthor.cumsum, 2),

              diff.papers.3.years = window.lag.computation(cumulative.papers, 3),
              diff.cumulative.coauthors.3.years = window.lag.computation(coauthor.cumsum, 3),

              diff.papers.4.years = window.lag.computation(cumulative.papers, 4),
              diff.cumulative.coauthors.4.years = window.lag.computation(coauthor.cumsum, 4),

              diff.papers.5.years = window.lag.computation(cumulative.papers, 5),
              diff.cumulative.coauthors.5.years = window.lag.computation(coauthor.cumsum, 5)
           )   
            
    col.names.not.na <- colnames(all.t)
  
    for (col in col.names.not.na) {
        if (sum(is.na(all.t[,col])) > 0) {
          all.t[is.na(all.t[,col]),col] <- 0
        }
    }
    

    all.t <- all.t %>%
        group_by(Node) %>% 
        arrange(Node, Time) %>% 
        mutate(    
              diff.2.papers = diff.papers - lag(diff.papers),
              diff.2.coauthors = diff.coauthors - lag(diff.coauthors),
              diff.2.cumulative.coauthors = diff.cumulative.coauthors - lag(diff.cumulative.coauthors),

              diff.2.papers.2.years = window.lag.computation(diff.papers.2.years, 2),
              diff.2.cumulative.coauthors.2.years = window.lag.computation(diff.cumulative.coauthors.2.years, 2),

              diff.2.papers.3.years = window.lag.computation(diff.papers.3.years, 3),
              diff.2.cumulative.coauthors.3.years = window.lag.computation(diff.cumulative.coauthors.3.years, 3),

              diff.2.papers.4.years = window.lag.computation(diff.papers.4.years, 4),
              diff.2.cumulative.coauthors.4.years = window.lag.computation(diff.cumulative.coauthors.4.years, 4),
                
              diff.2.papers.5.years = window.lag.computation(diff.papers.5.years, 5),
              diff.2.cumulative.coauthors.5.years = window.lag.computation(diff.cumulative.coauthors.5.years, 5),
            ) 
    
    # Remove extra lines after of authors' publication end
    all.t <- all.t %>% 
        group_by(Node, Time) %>%
        filter(
            (cumulative.papers != all.papers)
            |
            (Time == first.time.all.papers)
        )

    col.names.not.na <- colnames(all.t)
  
    for (col in col.names.not.na) {
        if (sum(is.na(all.t[,col])) > 0) {
          all.t[is.na(all.t[,col]),col] <- 0
        }
    }

  
  all.t
}

In [7]:
read.authors.features <- function(processed.dir, models, min.publications, window.begin=2000, window.end=2022) {
  window.begin <- window.begin - 2000 + 1
  window.end <- window.end - 2000 + 1
  tips.for.all.models  <- data.frame(matrix(ncol = 2, nrow = 0))
  colnames(tips.for.all.models ) <- c("Node", "Time")
  for (model in models) {
      print(paste0("Reading model ", model))
      tipness.files.names <- list.files(processed.dir, pattern = paste0("tip_decomposition-model-", model, "-step-\\d+-partition-1$"))
  
      # Sort by step  
      tmp.sort <- str_match(tipness.files.names, "tip_decomposition-model-(\\d+)-step-(\\d+)-.*")
      tipness.files.names <- tipness.files.names[order(as.integer(tmp.sort[, 2]), as.integer(tmp.sort[, 3]))]
  
      # Include from window.start to window.end (includes extremes)
      # Exclude 2021, 2022
      
      tipness.files.names <- tipness.files.names[window.begin:window.end]
      tipness.files.names <- paste(processed.dir, "/",tipness.files.names, sep="")
  
      # Tip value
      all.t <- read.all.tip.files(tipness.files.names)
      all.t$Node <- as.integer(as.character(all.t$Node))
      colnames(all.t) <- c(
          "Time",
          "Node",
          paste0("Degree.model.", model, sep=""),
          paste0("Neighbors.model.", model, sep=""),
          paste0("Entropy.model.", model, sep="")
      )
      tips.for.all.models <- merge(tips.for.all.models, all.t, by = c("Node", "Time"), all = T)
  }
  
  # Coauthors over time
  coauthors.t <- read.coauthors.cumsum.over.time(processed.dir)
  
  #print(tips.for.all.models[1:5,c("Node", "Time")])
  #print(coauthors.t[1:5,c("author", "time")])
    
  tips.for.all.models <- merge(tips.for.all.models, coauthors.t, by.x = c("Node", "Time"), by.y = c("author", "time"))
    
  #print(tips.for.all.models[1:5,c("Node", "Time")])
  
  tips.for.all.models <- tips.for.all.models %>%
    group_by(Node) %>%
    mutate(all.publications = sum(Degree.model.4, na.rm=T)) %>%
    ungroup() %>%
    as.data.frame()
    
  # At least min.publications in the time frame
  tips.for.all.models <- tips.for.all.models[
      tips.for.all.models$all.publications >= min.publications,
  ]
    
  tips.for.all.models$all.publications <- NULL
    
  tips.for.all.models
}

In [8]:
save.plot <- function(
    p, 
    file, 
    p.dir="/tmp/papers/scientometrics/",
    h = 900
){
    f.name <- paste0(p.dir,file,".png")
    print(f.name)
    png(
        file=f.name, 
        units="px", 
        width=1700, 
        height=h, 
        res=300)
    plot(p)
    dev.off()
}

In [9]:
plot.authors.behavior <- function(cl, all.publications.years, title=NULL) {
    
    cluster.pubs <- all.publications.years[all.publications.years$Node %in% cl,]
    cluster.pubs$Node <- as.factor(cluster.pubs$Node)
    
    all.publications.years <- all.publications.years[all.publications.years$Node %in% cl, ]
    all.publications.years$Node <- as.factor(all.publications.years$Node)
    
    breaks <- seq(
        min(all.publications.years$Time),
        max(all.publications.years$Time),
        1
    )

    p1 <- ggplot(cluster.pubs, aes(x = Time, y = cumulative.papers, group=Node)) +
           geom_line(aes(colour=Node), size=1) +
           scale_x_continuous(breaks=breaks) +
           theme(
               legend.position = "none", 
               text = element_text(size=15),
           )+
           ylab("Articles") +
           xlab("Year")


    p3 <- ggplot(cluster.pubs, aes(x = Time, y = coauthor.cumsum, group=Node)) +
           geom_line(aes(colour=Node), size = 1) +
           scale_x_continuous(breaks=breaks) +
           theme(
               legend.position = "none", 
               text = element_text(size=15),
           )+
           ylab("Cummulative coauthors") +
           xlab("Year")
    
      
    p4 <- ggplot(all.publications.years, aes(x = Time, y = diff.papers, group=Node)) +
           geom_line(aes(colour=Node), size = 1) +
           scale_x_continuous(breaks=breaks) +
           theme(
               legend.position = "none", 
               text = element_text(size=15),
           )+
           ylab("RAGS") +
           xlab("Year")
    
    p5 <- ggplot(all.publications.years, aes(x = Time, y = diff.cumulative.coauthors, group=Node)) +
           geom_line(aes(colour=Node), size = 1) +
           scale_x_continuous(breaks=breaks) +
           theme(
               legend.position = "none", 
               text = element_text(size=15),
           )+
           ylab("RCGS") +
           xlab("Year")
    
    p6 <- ggplot(all.publications.years, aes(x = Time, y = diff.butterflies, group=Node)) +
           geom_line(aes(colour=Node), size = 1) +
           scale_x_continuous(breaks=breaks) +
           theme(
               legend.position = "none", 
               text = element_text(size=15),
           )+
           ylab("Difference") +
           xlab("Year")
    
  
    p7 <- ggplot(
                all.publications.years[
                    ,
                ], 
                aes(x = Time, y = diff.2.papers, group=Node)
           ) +
           geom_line(aes(colour=Node), size = 1) +
           scale_x_continuous(breaks=breaks) +
           theme(
               legend.position = "none", 
               text = element_text(size=15),
           )+
           ylab("RAGA") +
           xlab("Year")
    
    p8 <- ggplot(
               all.publications.years[
                   ,
               ], aes(x = Time, y = diff.2.cumulative.coauthors, group=Node)
           ) +
           geom_line(aes(colour=Node), size = 1) +
           scale_x_continuous(breaks=breaks) +
           theme(
               legend.position = "none", 
               text = element_text(size=15),
           )+
           ylab("RCGA") +
           xlab("Year")
    
    p9 <- ggplot(all.publications.years, aes(x = Time, y = diff.2.butterflies, group=Node)) +
           geom_line(aes(colour=Node), size = 1) +
           scale_x_continuous(breaks=breaks) +
           theme(
               legend.position = "none", 
               text = element_text(size=15),
           )+
           ylab("Acceleration") +
           xlab("Time")
    
    save.plot(p1, paste0(title, ".cummulative.publications"))
    plot(p1)
    save.plot(p3, paste0(title, ".cummulative.coauthors"))
    plot(p3)
    save.plot(p4, paste0(title, ".publications.speed"))
    plot(p4)
    save.plot(p5, paste0(title, ".coauthors.speed"))
    plot(p5)
    save.plot(p7, paste0(title, ".publications.acceleration"))
    plot(p7)
    save.plot(p8, paste0(title, ".coauthors.acceleration"))
    plot(p8)
    
}

In [10]:
plot.author.behavior.2 <- function(
    cl, 
    baseline.authors,
    all.publications.years, 
    features.names, 
    friendly.features.names, 
    author.set.for.reference.1=NULL,
    author.set.for.reference.2=NULL) {
    
    cluster.pubs <- all.publications.years[all.publications.years$Node %in% cl,]
    cluster.pubs$Node <- as.factor(cluster.pubs$Node)
    

    x <- lapply(features.names, function(f.name) {
        mean.feature <- all.publications.years[,c("Node", "Time", f.name)]
        mean.feature$Node <- -1
        names(mean.feature) <- c("Node", "Time", "feature")
        
        mean.feature <- mean.feature %>%
          filter(feature >= 0) %>%
          group_by(Node, Time) %>%
          summarize(
              feature = mean(feature)
          ) %>%
          ungroup() %>%
          as.data.frame()
        
        mean.feature$Node <- as.factor(-1)
        
        
        sel.pubs <- cluster.pubs[
            ,
            c("Node", "Time", f.name)
        ]
        names(sel.pubs) <- c("Node", "Time", "feature")
        
        plot.features <- bind_rows(mean.feature, sel.pubs)
        
        if (!is.null(author.set.for.reference.1)) {
            mean.feature.ref <- all.publications.years[
                all.publications.years$Node %in% author.set.for.reference.1,
                c("Node", "Time", f.name)
            ]
            names(mean.feature.ref) <- c("Node", "Time", "feature")
            mean.feature.ref <- mean.feature.ref %>%
              filter(feature >= 0) %>%
              group_by(Time) %>%
              summarize(
                  feature = mean(feature)
              ) %>%
              ungroup() %>%
              as.data.frame()
        
            mean.feature.ref$Node <- as.factor(-2)
            
            plot.features <- bind_rows(plot.features, mean.feature.ref)
        }
        
        if (!is.null(author.set.for.reference.2)) {
            mean.feature.ref <- all.publications.years[
                all.publications.years$Node %in% author.set.for.reference.2,
                c("Node", "Time", f.name)
            ]
            names(mean.feature.ref) <- c("Node", "Time", "feature")
            mean.feature.ref <- mean.feature.ref %>%
              filter(feature >= 0) %>%
              group_by(Time) %>%
              summarize(
                  feature = mean(feature)
              ) %>%
              ungroup() %>%
              as.data.frame()
        
            mean.feature.ref$Node <- as.factor(-3)
            
            plot.features <- bind_rows(plot.features, mean.feature.ref)
        }
        
        plot.features <- split(plot.features, plot.features$Node)
        
        colors <- c(
            "black",
            "skyBlue",
            "orange",
            "yellow"
        )
        
        linetypes <- c(
            "solid", 
            "dashed", 
            "dotdash",
            "longdash"
        )
        
        nm.nodes <- names(plot.features) 
        p1 <- ggplot(plot.features[[nm.nodes[1]]], aes_string(x = "Time", y = "feature")) +
           geom_line(colour=colors[1], linetype=linetypes[1], size=1)
        
        i <- 2
        for (nm in nm.nodes[2:length(nm.nodes)]) {
            p1 <- p1 +
               geom_line(
                   data=plot.features[[nm]], 
                   aes_string(
                       x = "Time", 
                       y = "feature"
                   ), 
                   colour=colors[i], 
                
                   linetype=linetypes[i], 
                   size=1
               )
             i <- i + 1
         }
         p1 <- p1 +
           theme(
               text = element_text(size=15),
           )+
           ylab(friendly.features.names[[f.name]]) +
           xlab("Year") +
           scale_x_continuous(
               breaks=seq(2010,2020,1)
           )
        save.plot(p1, paste("comparative.behavior.",cl,".",f.name, collapse=".", sep=""))
        plot(p1)
    })
    
}

In [11]:
plot.author.behavior.3 <- function(
    cl, 
    baseline.authors,
    all.publications.years, 
    features.names, 
    friendly.features.names
    ) {
    
    cluster.pubs <- all.publications.years[all.publications.years$Node %in% cl,]
    cluster.pubs$Node <- as.factor(cluster.pubs$Node)
    baseline.pubs <- all.publications.years[all.publications.years$Node %in% baseline.authors,]
    baseline.pubs$Node <- as.factor(baseline.pubs$Node)
        
    x <- lapply(features.names, function(f.name) {
        plot.features <- all.publications.years[
            c(cl, baseline.authors),
            c("Node", "Time", f.name)
        ]
        p1 <- ggplot(plot.features, aes_string(x = "Time", y = f.name, group="Node")) +
           geom_line(
               data = cluster.pubs,
               aes(color = Node),
               linetype="solid", 
               size=1
           ) +
           geom_line(
               data = baseline.pubs,
               linetype="dashed", 
               size=1,
               colour="#6F7378"
           )
        p1 <- p1 +
           theme(
               legend.position = "none", 
               text = element_text(size=15),
           )+
           ylab(friendly.features.names[[f.name]]) +
           xlab("Year") +
           scale_x_continuous(
               breaks=seq(2010,2020,1)
           )
        save.plot(p1, paste("anomalous.midrank.behavior.",f.name, collapse=".", sep=""))
        plot(p1)
    })
}

In [12]:
compute.author.entropy <- function(weighted.graph) {
   
    compute.entropy <- function(weights) {
        p <- weights / sum(weights)
        log.p <- log2(p)
        e <- sum(-p * log.p)
        e <- e / log2(sum(weights))
        1 - e
    }
    
    entropy <- weighted.graph %>% 
        group_by(author) %>%
        summarize(
            entropy.opposite = compute.entropy(weight)
        )
    as.data.frame(entropy)
}

In [13]:
compute.author.venue.engagement.summary <- function(
    author.venue.engagement.year, 
    min.publications.venue.year=5,
    min.sd=0
    ) {
    
    author.venue.engagement.year <- author.venue.engagement.year[author.minus.venue.context.zscore > 0,]
    
    author.venue.engagement.year <- author.venue.engagement.year %>%
        filter(
            weight >= min.publications.venue.year & 
            author.minus.venue.context.zscore > min.sd
        ) %>%
        group_by(author, time) %>%
        summarize(
            mean.author.minus.venue.context.zscore = mean(author.minus.venue.context.zscore),
            max.author.minus.venue.context.zscore = max(author.minus.venue.context.zscore)
        )
    
    author.mean.zscore <- author.venue.engagement.year %>%
        group_by(author) %>%
        summarize(
            mean.author.minus.venue.context.zscore = mean(mean.author.minus.venue.context.zscore),
            max.author.minus.venue.context.zscore = max(max.author.minus.venue.context.zscore),
            count.author.minus.venue.context.zscore.gte.sd.ref = n()
        )

    author.mean.zscore
}

In [14]:
compute.author.venue.engagement.no.summary <- function(author.venue.year) {

    setDT(author.venue.year)
    
    author.venue.year[
        , 
        venue.pubs.sum := sum(weight), 
        by = .(time, venue)
    ]
    
    author.venue.year[
        , 
        author.pubs.sum := sum(weight), 
        by = .(time, author)
    ]
    
    mean.pubs.year <- mean(author.venue.year$author.pubs.sum)
    
    print(mean.pubs.year)
    
    author.venue.year[
        , 
        venue.context.pubs.sum := sum(venue.pubs.sum), 
        by = .(time, author)
    ]
    
        
    author.venue.year[
        , 
        c(
            "venue.context.zscore",
            "author.weight.zscore"
        ) := list(
            (venue.pubs.sum - mean(venue.pubs.sum)) / sd(venue.pubs.sum),
            (weight - mean(weight)) / sd(weight)
        ),
        by = .(time, author)
    ]
    
    author.venue.year[
        , 
        c(
            "author.minus.venue.context.zscore"
        ) := list(
            author.weight.zscore - venue.context.zscore
        )
    ]
    
    author.venue.year[
        is.na(author.minus.venue.context.zscore), 
        author.minus.venue.context.zscore := 0
    ]
    
    author.venue.year[
        , 
        venue.share := (venue.pubs.sum / venue.context.pubs.sum)
    ]
    
    author.venue.year[
        , 
        author.share := (weight / venue.pubs.sum)
    ]
    
    author.venue.year[
        , 
        author.share.over.venue.share := (author.share / venue.share)
    ]
    
    author.venue.year
    
}

In [15]:
compute.author.venue.engagement <- function(
    author.venue.year,
    min.publications.venue.year=5,
    min.sd=3
) {

    setDT(author.venue.year)
    
    author.venue.year[
        , 
        venue.pubs.sum := sum(weight), 
        by = .(time, venue)
    ]
    
    author.venue.year[
        , 
        author.pubs.sum := sum(weight), 
        by = .(time, author)
    ]
    
    mean.pubs.year <- mean(author.venue.year$author.pubs.sum)
    
    print(mean.pubs.year)
    
    author.venue.year[
        , 
        venue.context.pubs.sum := sum(venue.pubs.sum), 
        by = .(time, author)
    ]
    
        
    author.venue.year[
        , 
        c(
            "venue.context.zscore",
            "author.weight.zscore"
        ) := list(
            (venue.pubs.sum - mean(venue.pubs.sum)) / sd(venue.pubs.sum),
            (weight - mean(weight)) / sd(weight)
        ),
        by = .(time, author)
    ]
    
    author.venue.year[
        , 
        c(
            "author.minus.venue.context.zscore"
        ) := list(
            author.weight.zscore - venue.context.zscore
        )
    ]
    
    author.venue.year[
        is.na(author.minus.venue.context.zscore), 
        author.minus.venue.context.zscore := 0
    ]
    
    author.venue.year[
        , 
        venue.share := (venue.pubs.sum / venue.context.pubs.sum)
    ]
    
    author.venue.year[
        , 
        author.share := (weight / venue.pubs.sum)
    ]
    
    author.venue.year[
        , 
        author.share.over.venue.share := (author.share / venue.share)
    ]
    
    compute.author.venue.engagement.summary(
        author.venue.year,
        min.publications.venue.year,
        min.sd
    )
    
}

In [16]:
normalize.values <- function(all.t) {
  # Normalize/standardize all columns
  normalize.cols <- c(
    "all.publications",
    "all.coauthors",
    "mean.diff.2.papers.up",
    "var.diff.2.papers.up",
    "mean.diff.2.papers.abs",
    "var.diff.2.papers.abs",
    "mean.diff.2.coauthors.up",
    "var.diff.2.coauthors.up",
    "mean.diff.2.coauthors.abs",
    "var.diff.2.coauthors.abs",
    "mean.diff.2.cumulative.coauthors.up",
    "var.diff.2.cumulative.coauthors.up",
    "mean.diff.2.cumulative.coauthors.abs",
    "var.diff.2.cumulative.coauthors.abs",
    
    "mean.diff.2.papers.2.years.up",
    "var.diff.2.papers.2.years.up",
    "mean.diff.2.papers.2.years.abs",
    "var.diff.2.papers.2.years.abs",
    "mean.diff.2.cumulative.coauthors.2.years.up",
    "var.diff.2.cumulative.coauthors.2.years.up",
    "mean.diff.2.cumulative.coauthors.2.years.abs",
    "var.diff.2.cumulative.coauthors.2.years.abs",
    
    "mean.diff.2.papers.3.years.up",
    "var.diff.2.papers.3.years.up",
    "mean.diff.2.papers.3.years.abs",
    "var.diff.2.papers.3.years.abs",
    "mean.diff.2.cumulative.coauthors.3.years.up",
    "var.diff.2.cumulative.coauthors.3.years.up",
    "mean.diff.2.cumulative.coauthors.3.years.abs",
    "var.diff.2.cumulative.coauthors.3.years.abs",
    
    "mean.diff.2.papers.4.years.up",
    "var.diff.2.papers.4.years.up",
    "mean.diff.2.papers.4.years.abs",
    "var.diff.2.papers.4.years.abs",
    "mean.diff.2.cumulative.coauthors.4.years.up",
    "var.diff.2.cumulative.coauthors.4.years.up",
    "mean.diff.2.cumulative.coauthors.4.years.abs",
    "var.diff.2.cumulative.coauthors.4.years.abs",
    
    "mean.diff.2.papers.5.years.up",
    "var.diff.2.papers.5.years.up",
    "mean.diff.2.papers.5.years.abs",
    "var.diff.2.papers.5.years.abs",
    "mean.diff.2.cumulative.coauthors.5.years.up",
    "var.diff.2.cumulative.coauthors.5.years.up",
    "mean.diff.2.cumulative.coauthors.5.years.abs",
    "var.diff.2.cumulative.coauthors.5.years.abs",
    
    
    "mean.diff.papers",
    "var.diff.papers",
    "mean.diff.coauthors",
    "var.diff.coauthors",
    "mean.diff.cumulative.coauthors",
    "var.diff.cumulative.coauthors",
    "mean.diff.papers.2.years",
    "var.diff.papers.2.years",
    "mean.diff.cumulative.coauthors.2.years",
    "var.diff.cumulative.coauthors.2.years",
    "mean.diff.papers.3.years",
    "var.diff.papers.3.years",
    "mean.diff.cumulative.coauthors.3.years",
    "var.diff.cumulative.coauthors.3.years",
    "mean.diff.papers.4.years",
    "var.diff.papers.4.years",
    "mean.diff.cumulative.coauthors.4.years",
    "var.diff.cumulative.coauthors.4.years",
    "mean.diff.papers.5.years",
    "var.diff.papers.5.years",
    "mean.diff.cumulative.coauthors.5.years",
    "var.diff.cumulative.coauthors.5.years"
    
  )
  
  # Standardize
  for (col in normalize.cols) {
    new.col.name <- paste0("standardized.", col, sep="")
    all.t[, new.col.name] <- scale(all.t[, col])
    
    # Set NaN's as 0
    all.t[, new.col.name] <- sapply(all.t[, new.col.name], function(val) {
      if (is.nan(val)) {
        val <- 0
      }
      val
    })
  }
  all.t
}

In [17]:
compute.author.features.vector.gt.mean <- function(
    all.t, 
    author.index = NULL, 
    min.publications, 
    weighted.graph, 
    author.venue.year,
    min.publications.venue.year = 5,
    min.sd = 3
) {

  window.mean <- function(data, multiple) {
    if (length(data) >= multiple) {
        return(mean(data[seq(multiple, length(data), multiple)]))
    } else {
        return(0)
    }
  }
    
  window.var <- function(data, multiple) {
    if (length(data) >= 2*multiple) {
        return(var(data[seq(multiple, length(data), multiple)]))
    } else {
        return(0)
    }
  }
    
  window.mean.abs <- function(data, multiple) {
    if (length(data) >= multiple) {
        return(mean(abs(data[seq(multiple, length(data), multiple)])))
    } else {
        return(0)
    }
  }
    
  window.var.abs <- function(data, multiple) {
    if (length(data) >= 2*multiple) {
        return(var(abs(data[seq(multiple, length(data), multiple)])))
    } else {
        return(0)
    }
  }
        

  window.mean.up <- function(data, multiple) {
    if (length(data) >= multiple) {
        indices <- seq(multiple, length(data), multiple)
        data.indices <- data[indices]
        return(
            mean(
                data[indices[data.indices >= 0]]
            )
        )
    } else {
        return(0)
    }
  }
    
  window.var.up <- function(data, multiple) {
        if (length(data) >= 2*multiple) {
            indices <- seq(multiple, length(data), multiple)
            data.indices <- data[indices]
            data <- data[indices[data.indices >= 0]]
            if (length(data) > 1) {
                return(
                    var(
                        data
                    )
                )
            } else {
                return (0)
            }
        } else {
            return(0)
        }
    }
       

  all.t <- compute.derivatives.in.time(all.t, min.publications)

  if (!is.null(author.index)) {
    all.t$Node.name <- author.index[as.character(all.t$Node),]$author
  } else {
    all.t$Node.name <- as.factor(all.t$Node)
  }
    
  academic.birth <- compute.academic.birth(all.t)
  
  columns.names.diff.1 <- c(
      "diff.papers",
      "diff.coauthors", 
      "diff.cumulative.coauthors",
      
      "diff.papers.2.years", 
      "diff.coauthors.2.years", 
      "diff.cumulative.coauthors.2.years",
      
      "diff.papers.3.years", 
      "diff.coauthors.3.years", 
      "diff.cumulative.coauthors.3.years",
      
      "diff.papers.4.years", 
      "diff.coauthors.4.years", 
      "diff.cumulative.coauthors.4.years",
      
      "diff.papers.5.years", 
      "diff.coauthors.5.years", 
      "diff.cumulative.coauthors.5.years"
  )
  columns.names.diff.2 <- c(
      "diff.2.papers", 
      "diff.2.coauthors", 
      "diff.2.cumulative.coauthors",
      
      "diff.2.papers.2.years", 
      "diff.2.cumulative.coauthors.2.years",
      
      "diff.2.papers.3.years", 
      "diff.2.cumulative.coauthors.3.years",
      
      "diff.2.papers.4.years", 
      "diff.2.cumulative.coauthors.4.years",
      
      "diff.2.papers.5.years", 
      "diff.2.cumulative.coauthors.5.years"
      
  )

  abs.columns.names.diff.2 <- sapply(columns.names.diff.2, function(val) {
      paste0("abs.", val, sep="")
  })
  
  columns.names <- c("Node", "Node.name", columns.names.diff.1, columns.names.diff.2)

  # Absolute values for 2nd derivative
  for (col in columns.names.diff.2) {
    all.t[,paste0("abs.",col, sep="")] <- abs(all.t[,col])
  }
    
  all.t <- all.t %>% 
    group_by(Node, Node.name) %>% 
    arrange(Node, Node.name, Time, .by_group=TRUE) %>%
    summarise(
      all.publications = max(cumulative.papers),
      all.coauthors = max(coauthor.cumsum),
      
      mean.diff.2.papers.up = mean(diff.2.papers[diff.2.papers >= 0]),
      var.diff.2.papers.up = var(diff.2.papers[diff.2.papers >= 0]),

      mean.diff.2.papers.abs = mean(abs.diff.2.papers),
      var.diff.2.papers.abs = var(abs.diff.2.papers),
      
      mean.diff.2.coauthors.up = mean(diff.2.coauthors[diff.2.coauthors >= 0]),
      var.diff.2.coauthors.up = var(diff.2.coauthors[diff.2.coauthors >= 0]),
      
      mean.diff.2.coauthors.abs = mean(abs.diff.2.coauthors),
      var.diff.2.coauthors.abs = var(abs.diff.2.coauthors),
      
      mean.diff.2.cumulative.coauthors.up = mean(diff.2.cumulative.coauthors[diff.2.cumulative.coauthors >= 0]),
      var.diff.2.cumulative.coauthors.up = var(diff.2.cumulative.coauthors[diff.2.cumulative.coauthors >= 0]),  
        
      mean.diff.2.cumulative.coauthors.abs = mean(abs.diff.2.cumulative.coauthors),
      var.diff.2.cumulative.coauthors.abs = var(abs.diff.2.cumulative.coauthors),
      
      
      
      
      
      
      
      
      mean.diff.2.papers.2.years.up = window.mean.up(diff.2.papers.2.years, 2),
      var.diff.2.papers.2.years.up = window.var.up(diff.2.papers.2.years, 2),
        
      mean.diff.2.papers.2.years.abs = window.mean.abs(abs.diff.2.papers.2.years, 2),
      var.diff.2.papers.2.years.abs = window.var.abs(abs.diff.2.papers.2.years, 2),
      
      mean.diff.2.cumulative.coauthors.2.years.up = window.mean.up(diff.2.cumulative.coauthors.2.years, 2),
      var.diff.2.cumulative.coauthors.2.years.up = window.var.up(diff.2.cumulative.coauthors.2.years, 2),  
        
      mean.diff.2.cumulative.coauthors.2.years.abs = window.mean.abs(abs.diff.2.cumulative.coauthors.2.years, 2),
      var.diff.2.cumulative.coauthors.2.years.abs = window.var.abs(abs.diff.2.cumulative.coauthors.2.years, 2),
      
      
      
      
      
      mean.diff.2.papers.3.years.up = window.mean.up(diff.2.papers.3.years, 3),
      var.diff.2.papers.3.years.up = window.var.up(diff.2.papers.3.years, 3),
        
      mean.diff.2.papers.3.years.abs = window.mean.abs(abs.diff.2.papers.3.years, 3),
      var.diff.2.papers.3.years.abs = window.var.abs(abs.diff.2.papers.3.years, 3),
      
      mean.diff.2.cumulative.coauthors.3.years.up = window.mean.up(diff.2.cumulative.coauthors.3.years, 3),
      var.diff.2.cumulative.coauthors.3.years.up = window.var.up(diff.2.cumulative.coauthors.3.years, 3),  
        
      mean.diff.2.cumulative.coauthors.3.years.abs = window.mean.abs(abs.diff.2.cumulative.coauthors.3.years, 3),
      var.diff.2.cumulative.coauthors.3.years.abs = window.var.abs(abs.diff.2.cumulative.coauthors.3.years, 3),
      
      
      
      mean.diff.2.papers.4.years.up = window.mean.up(diff.2.papers.4.years, 4),
      var.diff.2.papers.4.years.up = window.var.up(diff.2.papers.4.years, 4),
        
      mean.diff.2.papers.4.years.abs = window.mean.abs(abs.diff.2.papers.4.years, 4),
      var.diff.2.papers.4.years.abs = window.var.abs(abs.diff.2.papers.4.years, 4),
      
      mean.diff.2.cumulative.coauthors.4.years.up = window.mean.up(diff.2.cumulative.coauthors.4.years, 4),
      var.diff.2.cumulative.coauthors.4.years.up = window.var.up(diff.2.cumulative.coauthors.4.years, 4),  
        
      mean.diff.2.cumulative.coauthors.4.years.abs = window.mean.abs(abs.diff.2.cumulative.coauthors.4.years, 4),
      var.diff.2.cumulative.coauthors.4.years.abs = window.var.abs(abs.diff.2.cumulative.coauthors.4.years, 4),
      
      
      
      mean.diff.2.papers.5.years.up = window.mean.up(diff.2.papers.5.years, 5),
      var.diff.2.papers.5.years.up = window.var.up(diff.2.papers.5.years, 5),
        
      mean.diff.2.papers.5.years.abs = window.mean.abs(abs.diff.2.papers.5.years, 5),
      var.diff.2.papers.5.years.abs = window.var.abs(abs.diff.2.papers.5.years, 5),
      
      mean.diff.2.cumulative.coauthors.5.years.up = window.mean.up(diff.2.cumulative.coauthors.5.years, 5),
      var.diff.2.cumulative.coauthors.5.years.up = window.var.up(diff.2.cumulative.coauthors.5.years, 5),  
        
      mean.diff.2.cumulative.coauthors.5.years.abs = window.mean.abs(abs.diff.2.cumulative.coauthors.5.years, 5),
      var.diff.2.cumulative.coauthors.5.years.abs = window.var.abs(abs.diff.2.cumulative.coauthors.5.years, 5),
      
      
      
      
      mean.diff.papers = mean(diff.papers),
      var.diff.papers = var(diff.papers),
      
      mean.diff.coauthors = mean(diff.coauthors),
      var.diff.coauthors = var(diff.coauthors),
      
      mean.diff.cumulative.coauthors = mean(diff.cumulative.coauthors),
      var.diff.cumulative.coauthors = mean(diff.cumulative.coauthors),


      mean.diff.papers.2.years = window.mean(diff.papers.2.years, 2),
      var.diff.papers.2.years = window.var(diff.papers.2.years, 2),
      
      mean.diff.cumulative.coauthors.2.years = window.mean(diff.cumulative.coauthors.2.years, 2),
      var.diff.cumulative.coauthors.2.years = window.var(diff.cumulative.coauthors.2.years, 2),

        
        
      mean.diff.papers.3.years = window.mean(diff.papers.3.years, 3),
      var.diff.papers.3.years = window.var(diff.papers.3.years, 3),
      
      mean.diff.cumulative.coauthors.3.years = window.mean(diff.cumulative.coauthors.3.years, 3),
      var.diff.cumulative.coauthors.3.years = window.var(diff.cumulative.coauthors.3.years, 3),
        
        
      
      mean.diff.papers.4.years = window.mean(diff.papers.4.years, 4),
      var.diff.papers.4.years = window.var(diff.papers.4.years, 4),
      
      mean.diff.cumulative.coauthors.4.years = window.mean(diff.cumulative.coauthors.4.years, 4),
      var.diff.cumulative.coauthors.4.years = window.var(diff.cumulative.coauthors.4.years, 4),
        
        
        
      
      mean.diff.papers.5.years = window.mean(diff.papers.5.years, 5),
      var.diff.papers.5.years = window.var(diff.papers.5.years, 5),
        
      mean.diff.cumulative.coauthors.5.years = window.mean(diff.cumulative.coauthors.5.years, 5),
      var.diff.cumulative.coauthors.5.years = window.var(diff.cumulative.coauthors.5.years, 5)

      
      
    ) %>%
    ungroup() %>% 
    as.data.frame()
  
  all.t <- all.t[all.t$all.publications > min.publications,]
  
  entropy <- compute.author.entropy(weighted.graph)
    
  author.venue.engagement <- compute.author.venue.engagement(
      author.venue.year,
      min.publications.venue.year,
      min.sd
  )
    
  all.t <- merge(all.t, entropy, by.x="Node", by.y="author", all.x=T)
    
  all.t <- merge(all.t, academic.birth, by.x="Node", by.y="Node", all.x=T)
    
  all.t <- merge(all.t, author.venue.engagement, by.x="Node", by.y="author", all.x=T)
    
  all.t[is.na(all.t$mean.author.minus.venue.context.zscore),]$mean.author.minus.venue.context.zscore <- 0
  
  all.t
}

In [18]:
corr.high <- function(data=df,sig=0.5,method="pearson"){
  #convert data to numeric in order to run correlations
  #convert to factor first to keep the integrity of the data - each value will become a number rather than turn into NA
  df_cor <- data %>% mutate_if(is.character, as.factor)
  df_cor <- df_cor %>% mutate_if(is.factor, as.numeric)  #run a correlation and drop the insignificant ones
  corr <- cor(df_cor, method=method)
  #prepare to drop duplicates and correlations of 1     
  corr[lower.tri(corr,diag=TRUE)] <- NA 
  #drop perfect correlations
  corr <- abs(corr)
  corr[corr == 1] <- NA   #turn into a 3-column table
  corr <- as.data.frame(as.table(corr))
  #remove the NA values from above 
  corr <- na.omit(corr)   #select significant values  
  corr <- subset(corr, abs(Freq) > sig) 
  #sort by highest correlation
  corr <- corr[order(-abs(corr$Freq)),]   #print table
  #print(corr)  #turn corr back into matrix in order to plot with corrplot
  mtx_corr <- reshape2::acast(corr, Var1~Var2, value.var="Freq")
  
  #plot correlations visually
  #corrplot(mtx_corr, is.corr=FALSE, tl.col="black", na.label=" ")
  corr
}

corr.low <- function(data=df,sig=0.5,method="pearson"){
  #convert data to numeric in order to run correlations
  #convert to factor first to keep the integrity of the data - each value will become a number rather than turn into NA
  df_cor <- data %>% mutate_if(is.character, as.factor)
  df_cor <- df_cor %>% mutate_if(is.factor, as.numeric)  #run a correlation and drop the insignificant ones
  corr <- cor(df_cor, method=method)
  #prepare to drop duplicates and correlations of 1     
  corr[lower.tri(corr,diag=TRUE)] <- NA 
  #drop perfect correlations
  corr <- abs(corr)
  corr[corr == 1] <- NA   #turn into a 3-column table
  corr <- as.data.frame(as.table(corr))
  #remove the NA values from above 
  corr <- na.omit(corr)   #select significant values  
  corr <- subset(corr, abs(Freq) < sig) 
  #sort by highest correlation
  corr <- corr[order(abs(corr$Freq)),]   #print table
  print(corr)  #turn corr back into matrix in order to plot with corrplot
  mtx_corr <- reshape2::acast(corr, Var1~Var2, value.var="Freq")
  
  #plot correlations visually
  #corrplot(mtx_corr, is.corr=FALSE, tl.col="black", na.label=" ")
  corr
}

In [19]:
library(repr)

## Journal Publication Analysis

### Code - Read data

In [20]:
venue.type <- "journal"

In [21]:
# base_data_dir <- "[your local repository base folder]"
base_data_dir <- "/home/edre/doutorado/projeto/scientometrics-resources/"
data.dir <- paste0(base_data_dir,"dblp/")
graphs.dir <- paste0(data.dir, venue.type)
processed.dir <- paste0(data.dir, "/tip-decompositions-", venue.type)
min.publications <- 50
authors.file <- paste0(data.dir, "author-index")

window.begin <- 2010
window.end <- 2020

# Publications in one year only -> model = 4
# It depends on the precomputed data
models <- c(4)


In [22]:
author.index <- read.author.index(authors.file)

In [23]:
all.publications.years <- read.authors.features(processed.dir, models, min.publications, window.begin, window.end)
all.publications.years$Time <- all.publications.years$Time + 2000

[1] "Reading model 4"


#### Author - Venue by year

In [24]:
author.venue.files <- "author-venue/journal/"
author.venue.files <- paste0(data.dir, author.venue.files)
author.venue.year <- read.graph.author.venue.all.steps.by.year(author.venue.files, window.begin, window.end)
length(unique(author.venue.year$venue))

[1] 2835

#### Author - venue global

In [25]:
author.venue.weighted.edge.list <- read.graph.author.venue.all.steps(author.venue.files, window.begin, window.end)

### Features extraction

#### Code - features cmputation

In [26]:
all.t <- compute.derivatives.in.time(all.publications.years, min.publications)

In [27]:
write.csv(all.t, "/tmp/authors-features.csv")

In [28]:
gc(verbose=TRUE)

,used,(Mb),gc trigger,(Mb),max used,(Mb)
Ncells,11859947,633.4,32210250,1720.3,32210250,1720.3
Vcells,61218974,467.1,113252846,864.1,117746455,898.4


In [29]:
total.pub.per.author.tmp <- author.venue.year %>%
    group_by(author) %>% 
    mutate(total.articles = sum(weight))
total.pub.per.author.tmp <- total.pub.per.author.tmp[
    total.pub.per.author.tmp$total.articles >= min.publications,
]

In [30]:
ecdf(total.pub.per.author.tmp$weight)(6)

[1] 0.9897033

In [31]:
features <- compute.author.features.vector.gt.mean(
    all.publications.years, 
    author.index = author.index, 
    min.publications = min.publications,
    weighted.graph = author.venue.weighted.edge.list,
    author.venue.year = author.venue.year,
    min.publications.venue.year=6,
    min.sd=0
)

[1] 2.690893


In [32]:
authors.papers.graph <- read.graph.author.publication.all.steps(graphs.dir, window.begin, window.end)

Warning message in file(file, "rt"):
“não foi possível abrir o arquivo '/home/edre/doutorado/projeto/scientometrics-resources/dblp/journal/NA': Arquivo ou diretório inexistente”


ERROR: Error in file(file, "rt"): não é possível abrir a conexão


In [ ]:
head(authors.papers.graph)

In [ ]:
authors.papers.graph <- authors.papers.graph[
    authors.papers.graph$author %in% features$Node,
]
print(length(unique(authors.papers.graph$author)))
print(length(unique(authors.papers.graph$paper)))
x <- author.venue.weighted.edge.list[
    author.venue.weighted.edge.list$author %in% features$Node,
]
print(length(unique(x$venue)))
y <- authors.papers.graph %>% 
    group_by(author) %>% 
    summarize(num.papers = length(unique(paper)))
print(mean(y$num.papers))

In [ ]:
y <- x %>% group_by(author) %>% summarize(num.venues = length(unique(venue)))
mean(y$num.venues)

### Author count in our analysis

In [ ]:
nrow(features)
nrow(all.publications.years)

### Distribution for each feature

In [ ]:
features.author.paper.up <- c(

    mean.diff.papers = "RAGS mean",
    var.diff.papers = "RAGS standard deviation",
    mean.diff.papers.2.years = "Mean",
    var.diff.papers.2.years = "Standard deviation",
    mean.diff.papers.3.years = "Mean",
    var.diff.papers.3.years = "Variance",
    mean.diff.papers.4.years = "Mean",
    var.diff.papers.4.years = "Variance",
    mean.diff.papers.5.years = "Mean",
    var.diff.papers.5.years = "Variance",

    mean.diff.cumulative.coauthors = "RCGS speed mean",
    var.diff.cumulative.coauthors = "RCGS standard deviation",
    mean.diff.cumulative.coauthors.2.years = "Mean",
    var.diff.cumulative.coauthors.2.years = "Standard deviation",
    mean.diff.cumulative.coauthors.3.years = "Mean",
    var.diff.cumulative.coauthors.3.years = "Variance",
    mean.diff.cumulative.coauthors.4.years = "Mean",
    var.diff.cumulative.coauthors.4.years = "Variance",
    mean.diff.cumulative.coauthors.5.years = "Mean",
    var.diff.cumulative.coauthors.5.years = "Variance",


    mean.diff.2.papers.up = "RAGA mean",
    var.diff.2.papers.up = "RAGA standard deviation",
    mean.diff.2.papers.2.years.up = "Mean",
    var.diff.2.papers.2.years.up = "Standard deviation",
    mean.diff.2.papers.3.years.up = "Mean",
    var.diff.2.papers.3.years.up = "Variance",
    mean.diff.2.papers.4.years.up = "Mean",
    var.diff.2.papers.4.years.up = "Variance",
    mean.diff.2.papers.5.years.up = "Mean",
    var.diff.2.papers.5.years.up = "Variance",


    mean.diff.2.cumulative.coauthors.up = "RCGA mean",
    var.diff.2.cumulative.coauthors.up = "RCGA standard deviation",
    mean.diff.2.cumulative.coauthors.2.years.up = "Mean",
    var.diff.2.cumulative.coauthors.2.years.up = "Variance",
    mean.diff.2.cumulative.coauthors.3.years.up = "Mean",
    var.diff.2.cumulative.coauthors.3.years.up = "Variance",
    mean.diff.2.cumulative.coauthors.4.years.up = "Mean",
    var.diff.2.cumulative.coauthors.4.years.up = "Variance",
    mean.diff.2.cumulative.coauthors.5.years.up = "Mean",
    var.diff.2.cumulative.coauthors.5.years.up = "Variance",
    
    entropy.opposite = "1 - entropy",
    mean.author.minus.venue.context.zscore = "SURPLUS"
)

In [ ]:
selected.features <- names(features.author.paper.up)

In [ ]:
length(selected.features)

### Anomaly ranking computation

#### Code - ranking computation

In [ ]:
compute.outlier.rank <- function (df) {
    rank(df, ties="min")
}

In [ ]:
outlierness <- lapply(selected.features, function(feat){
    feature1 <- feat
    outlier.rank <- compute.outlier.rank(features[,feat])
    list(
        feature1 = feature1,
        outlier.rank = outlier.rank
    )
})

#### Plot for the top 10 anomalous authors regarding each individual feature

In [ ]:
features.to.plot <- c(
    "diff.papers",
    "diff.2.papers",
    "diff.cumulative.coauthors",
    "diff.2.cumulative.coauthors"
)
x <- lapply(outlierness, function(anom) {
    print(anom$feature1)
    if (
        anom$feature1 %in% paste("mean.",features.to.plot, sep="") || 
        anom$feature1 %in% paste("mean.",features.to.plot,".up", sep="")
    ) {
        print(anom$feature1)
        print(features[order(-anom$outlier.rank),]$Node[1:10])
        plot.authors.behavior(
            features[order(-anom$outlier.rank),]$Node[1:10],
            all.t, 
            paste0("anomalous.for.feature.", anom$feature1))
    }
})

### Discriminative features

In [ ]:
plots.dir <- "/tmp/papers/scientometrics/"

In [ ]:
#head(all.publications.years$Degree.model.4, n=10)
ecdfs <- list()
for (year in unique(all.publications.years$Time)) {
    df.year <- all.publications.years[all.publications.years$Time == year,]
    df.year.data <- df.year$Degree.model.4
    df.year.data <- df.year.data[!is.na(df.year.data)]
    
    min.val <- min(df.year.data, na.rm=T)
    max.val <- max(df.year.data, na.rm=T)
    
    step <- 1#(max.val-min.val)/100
    
    slots <- seq(min.val, max.val+step, by=step)

    ecdf.data <- sapply(slots, function(slt) {
        sum(df.year.data <= slt) / length(df.year.data)
    })
    ecdfs[[as.character(year)]] <- data.frame(x=c(0,slots), y=c(0,ecdf.data))
}

In [ ]:
years <- sort(names(ecdfs))

colfunc <- colorRampPalette(c("red","black"))
linetype <- c("dotdash", "solid")
colors <- colfunc(length(names(ecdfs)))
names(colors) <- sort(names(ecdfs))

p <- ggplot()

years <- c("2010", "2020")

names(linetype) <- sort(years)

x.labels <- c()
min.99 <- list()

# Number of papers covering 99% of the population in 2010
NUM.PUBS.99.2010 <- 19

for (year in years) {
    df <- ecdfs[[year]]
    print(paste("year: ", year, ", 95%", sep=""))
    print(df[sum(df$y < 0.95),])
    print(
        max(
            all.publications.years[all.publications.years$Time == year,]$Degree.model.4,
            na.rm=T
        )
    )
    print(paste("year: ", year, ", 99%", sep=""))
    print(df[sum(df$y < 0.99),])
        
    print(paste("year: ", year, ", author >= 19 articles", sep=""))
    print(df[df$x >= 19,]$y[1])
    
    min.99[[year]] <- df[sum(df$y < 0.99),]$x
    x.labels <- c(x.labels, max(ecdfs[[year]]$x))
    p <- p +
        geom_line(
            data=ecdfs[[year]], 
            aes(x=x, y=y), 
            color=colors[year], 
            linetype=linetype[year],
            size=1
        ) +
        geom_vline(
            xintercept=max(ecdfs[[year]]$x), 
            color=colors[year], 
            linetype=linetype[year]
        )
}
breaks <- c(1, 10, max(ecdfs[["2010"]]$x), max(ecdfs[["2020"]]$x))
p <- p +
    geom_text(
         data=data.frame(x=x.labels, label=years),
         mapping = aes(x = x,
                       y = 0,
                       angle = 90,
                       hjust = -1,
                       vjust = -1,
                       label=years
         ),
        size=6
    ) +
    scale_x_continuous(
       #trans="log10",
        limits=c(NA,140),
        breaks=breaks
    ) +
#    labs(
#        y="",
#        x="Articles"
#    ) +
    theme(
       #legend.position = "none", 
       text = element_text(size=15),
       #axis.title.x="Articles per athor",# element_blank(),
       #axis.title.y="ECDF",#element_blank(),
    ) +
    labs(
      x = "Articles per author",
      y = "ECDF"
    )
    save.plot(p, "ecdf_2010_2020.png")
p

#### Most prolific authors by 2010

In [ ]:
df.2010 <- all.publications.years[all.publications.years$Time == 2010,]
df.2010 <- df.2010[!is.na(df.2010$Degree.model.4),]
df.2010 <- df.2010[df.2010$Degree.model.4 > min.99[["2010"]],]

prolific.2010 <- df.2010$Node

In [ ]:
length(prolific.2010)
nrow(all.publications.years[all.publications.years$Time == 2010,])
x <- all.publications.years %>% group_by(Node) %>% summarize(all.pubs = sum(Degree.model.4))
head(x[order(x$all.pubs),])

#### Most prolific authors by 2020

In [ ]:
df.2020 <- all.publications.years[all.publications.years$Time == 2020,]
df.2020 <- df.2020[!is.na(df.2020$Degree.model.4),]
df.2020.baseline <- df.2020[df.2020$Degree.model.4 > min.99[["2020"]],]

# Only top 100 authors
#df.2020 <- df.2020[order(-df.2020$Degree.model.4) <= 100,]

#head(df.2020)

prolific.2020 <- df.2020$Node
prolific.2020.baseline <- df.2020.baseline$Node
length(prolific.2020.baseline)

In [ ]:
names(df.2020[df.2020$Degree.model.4 > 120,])
df.2020[df.2020$Degree.model.4 > 120,c("Neighbors.model.4", "coauthor.cumsum")]

In [ ]:
nrow(df.2020)
print(
    length(
        unique(
            df.2020[
                df.2020$Degree.model.4 >= NUM.PUBS.99.2010,
            ]$Node
        )
    )
)

#### Venues birth year

In [ ]:
venue.birth <- author.venue.year %>%
    group_by(venue) %>%
    summarize(
        birth.year = min(time)
    )
venue.birth <- venue.birth[order(venue.birth$birth.year),]
venue.birth %>% group_by(birth.year) %>% summarize(count=n())

In [ ]:
venue.time <- unique(author.venue.year[,c("venue", "time")])
venue.time <- venue.time %>% 
    group_by(time) %>%
    summarize(count = n())

#### Exibe plots das features para um autor do meio da curva de ECDF e outro do topo

In [ ]:
baseline.authors <- prolific.2020.baseline

In [ ]:
features.names.1 <- c(
        diff.papers = "RAGS",
        diff.2.papers = "RAGA",
        diff.cumulative.coauthors = "RCGS",
        diff.2.cumulative.coauthors = "RCGA" 
)

authors.feats <- lapply(names(features.names.1), function(col) {
    feature.name <- paste0("mean.", col)
    if (grepl("diff.2", feature.name, fixed = TRUE)) {
        feature.name <- paste0(feature.name, ".up")
    }
    print(feature.name)
    max.col <- which.max(features[,feature.name])
    author <- features[max.col, "Node"]
    feature <- all.t[all.t$Node == author & all.t$Time == 2020, col]
    
    max.feature <- max(all.t[all.t$Node == author, col])
    min.feature <- min(all.t[all.t$Node == author, col])
    
    print(paste(feature.name, author, ", last: ", feature, ", max: ", max.feature, ", min: ", min.feature, sep=", "))
    
    plot.author.behavior.2(
        author, 
        baseline.authors,
        all.t, 
        col, 
        features.names.1,
        baseline.authors
    )
})


features.names <- c(
        "mean.diff.papers",
        "var.diff.papers",
        "mean.diff.2.papers.up",
        "var.diff.2.papers.up",
        "entropy.opposite",
        "mean.diff.cumulative.coauthors",
        "var.diff.cumulative.coauthors",
        "mean.diff.2.cumulative.coauthors.up",
        "var.diff.2.cumulative.coauthors.up",
        "mean.author.minus.venue.context.zscore"
)

In [ ]:
authors.list <- c("40895")
plot.author.behavior.2(
    authors.list, 
    baseline.authors,
    all.t, 
    names(features.names.1), 
    features.names.1,
    baseline.authors
)

Plot the ECDF for each feature

In [ ]:
selected.for.plots <- sapply(outlierness, function(outlrns) {
    if (outlrns$feature1 %in% features.names) {
        print(outlrns$feature1)
        return (outlrns)
    } else {
        return (NA)
    }
})

selected.for.plots <- selected.for.plots[!is.na(selected.for.plots)]

x <- lapply (selected.for.plots, function(outlrns) {

    ecdf.data.population <- features[
        ,
        outlrns$feature1
    ]
    
    if (grepl("var.", outlrns$feature1, fixed = TRUE)) {
        ecdf.data.population <- sqrt(ecdf.data.population)
    }
    
    min.val.feature <- min(ecdf.data.population, na.rm=T)
    max.val.feature <- max(ecdf.data.population, na.rm=T)

    step <- (max.val.feature-min.val.feature)/100
    if (F) {
    if (step < 100) {
        step <- 10
    } else if (step < 1000) {
        step <- 100
    } else if (step < 10000) {
        step <- 1000
    }
    }
    
    print(outlrns$feature1)
    print(min(ecdf.data.population))
    print(max(ecdf.data.population))
    print(min.val.feature)
    print(max.val.feature)

    
    slots <- seq(min.val.feature, max.val.feature+step, by=step)

    x <- ecdf.data.population
    x <- x[!is.na(x)]
    ecdf.data <- sapply(slots, function(slt) {
        sum(x <= slt) / length(x)
    })
    
    # Index corresponding to 99% in ECDF
    THRESHOLD <- 0.99
    idx99 <- sum(ecdf.data < THRESHOLD)
    
    # Value corresponding to 99% in ECDF
    val99 <- slots[idx99]
    print(paste("val99: ", val99))
    
    # Number of occurrencies above the threshold
    ecdf.data.top.anoms <- features[
        which(ecdf.data.population >= val99)
        ,
        outlrns$feature1
    ]
    
    if (grepl("var.", outlrns$feature1, fixed = TRUE)) {
        ecdf.data.top.anoms <- sqrt(ecdf.data.top.anoms)
    }
    
    ecdf.top.anoms <- sapply(slots, function(slt) {
        sum(ecdf.data.top.anoms <= slt) / length(ecdf.data.top.anoms)
    })

    ecdf.data.prolific.2010 <- features[
        features$Node %in% baseline.authors#prolific.2010
        ,
        outlrns$feature1
    ]
    
    if (grepl("var.", outlrns$feature1, fixed = TRUE)) {
        ecdf.data.prolific.2010 <- sqrt(ecdf.data.prolific.2010)
    }
    
    ecdf.prolific.2010 <- sapply(slots, function(slt) {
        sum(ecdf.data.prolific.2010 <= slt) / length(ecdf.data.prolific.2010)
    })
    
    ecdf.val.prolific.99 <- ecdf.prolific.2010[idx99]

    data.ecdf.plot <- data.frame(x = slots, y1 = ecdf.data, y2 = ecdf.top.anoms, y3 = ecdf.prolific.2010)

    y.labels <- function(x) sprintf("%.2f", x)

    #print(data.ecdf.plot)
    #print(outlrns$feature1)
    if (!grepl("entropy", outlrns$feature1, fixed = TRUE)) {
        p <- plot(ggplot(data.ecdf.plot, aes(x=x, y=y1)) +
          geom_line(color = "black", size = 1, linetype="solid") +
          geom_line(data = data.ecdf.plot, aes(x=x, y=y2), color = "orange", size = 1, linetype="dotdash") +
          geom_line(data = data.ecdf.plot, aes(x=x, y=y3), color = "blue", size = 1, linetype="longdash") +
          #geom_line(data = data.ecdf.plot, aes(x=x, y=y3), color = "blue", size = 1, linetype="longdash") +
          scale_x_continuous(
              #trans="log10", 
              breaks=function(bob) {
                  min.val <- min(bob)
                  #min.val <- min.val %/% 10 * 10

                  max.val <- max(bob)
                  step <- (max.val - min.val) / 10
                  #print(paste("step: ",step))
                  #if (step < 10) step <- 10
                  #else if (step < 100) step <- 100
                  #else if (step < 1000) step <- 1000
                  #else if (step < 10000) step <- 10000
                  #print(paste("step: ",step))
                  brks <- seq(min.val, max.val, step)

                  pos99 <- sum(brks < val99)

                  if (abs(val99 - brks[pos99]) > 10 &&
                      abs(val99 - brks[pos99+1]) > 10) {

                      brks <- append(brks, val99, after=pos99)
                  } else {
                      if (abs(val99 - brks[pos99]) <= 10) {
                          brks[pos99] <- val99
                      } else {
                          brks[pos99+1] <- val99
                      }
                  }
                  
                  #print(paste("step: ", step))
                  #print(paste("brks: ", brks))
                  #print(paste("max: ", max.val))

                  if (max.val > 1.5) {
                      brks <- ceiling(brks)
                  } else {
                      brks <- brks[brks <= 1]
                      brks <- round(brks, 1)
                      if (!1 %in% brks) {
                          brks <- append(brks, 1)
                      }
                      print(brks)
                      brks
                  }
                  if (outlrns$feature1 == "mean.author.minus.venue.context.zscore") {
                  
                      pos99 <- sum(brks < val99)

                      if (abs(val99 - brks[pos99]) <= 10) {
                          brks[pos99] <- val99
                      } else {
                          brks[pos99+1] <- val99
                      }
                      brks <- round(brks, digits=1)
                  }
                  brks
                  
              }
          ) +
          scale_y_continuous(
              breaks=sort(c(seq(0, 1, length.out=5), ecdf.val.prolific.99)),
              labels=y.labels(sort(c(seq(0, 1, length.out=5), ecdf.val.prolific.99)))
          ) +
          geom_vline(xintercept=slots[idx99], linetype="dotted") + 
          geom_hline(yintercept=ecdf.val.prolific.99, linetype="dotted") + 
          theme(
              # axis.title.x=element_blank(),
              # axis.title.y=element_blank(),
              text = element_text(size=15),
              axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1)) +
          labs(
            x = features.author.paper.up[[outlrns$feature1]],
            y = "ECDF"
          )
        )
    } else {
        p <- plot(ggplot(data.ecdf.plot, aes(x=x, y=y1)) +
          geom_line(color = "black", size = 1, linetype="solid") +
          geom_line(data = data.ecdf.plot, aes(x=x, y=y2), color = "orange", size = 1, linetype="dotdash") +
          geom_line(data = data.ecdf.plot, aes(x=x, y=y3), color = "blue", size = 1, linetype="longdash") +
          #geom_line(data = data.ecdf.plot, aes(x=x, y=y3), color = "blue", size = 1, linetype="longdash") +
          scale_x_continuous(
              breaks=function(bob) {
                  min.val <- min(bob)
                  #min.val <- min.val %/% 10 * 10

                  max.val <- max(bob)
                  step <- (max.val - min.val) / 10
                  #print(paste("step: ",step))
                  #if (step < 10) step <- 10
                  #else if (step < 100) step <- 100
                  #else if (step < 1000) step <- 1000
                  #else if (step < 10000) step <- 10000
                  #print(paste("step: ",step))
                  brks <- seq(min.val, max.val, step)

                  pos99 <- sum(brks < val99)
                  brks <- append(brks, val99, after=pos99)

                  #if (abs(val99 - brks[pos99]) > 10 &&
                  #    abs(val99 - brks[pos99+1]) > 10) {

                  #    brks <- append(brks, val99, after=pos99)
                  #} else {
                  #    if (abs(val99 - brks[pos99]) <= 10) {
                  #        brks[pos99] <- val99
                  #    } else {
                  #        brks[pos99+1] <- val99
                  #    }
                  #}

                  #print(paste("step: ", step))
                  #print(paste("brks: ", brks))
                  #print(paste("max: ", max.val))

                  if (max.val > 1.5) {
                      ceiling(brks)
                  } else {
                      brks <- brks[brks <= 1]
                      brks <- round(brks, 1)
                      if (!1 %in% brks) {
                          brks <- append(brks, 1)
                      }
                      print(brks)
                      brks
                  }
              }
          ) +
          # scale_y_continuous(
          #    breaks=sort(c(seq(0, 1, length.out=5), ecdf.val.prolific.99)),
          #    labels= y.labels(sort(c(seq(0, 1, length.out=5), ecdf.val.prolific.99)))
          #) +
          geom_vline(xintercept=slots[idx99], linetype="dotted") +
          #geom_hline(yintercept=ecdf.val.prolific.99, linetype="dotted") + 
          theme(
              #axis.title.x=element_blank(),
              #axis.title.y=element_blank(),
              text = element_text(size=15),
              axis.text.x = element_text(
                  angle = 90, 
                  vjust = 0.5, 
                  hjust=1)) +
          labs(
            x = "1 - ENTROPY",
            y = "ECDF"
          )
        )
    }
    save.plot(p, paste0("ecdf.", outlrns$feature1))
})

#### Prune correlated ranks

In [ ]:
# Rename features
outlierness <- lapply(outlierness, function(scores) {
    scores$feature1 <- str_replace(scores$feature1, "standardized.", "")
    scores
})

In [ ]:
#acceleration.up <- list()
ranks <- lapply(outlierness, function(outl) {
    outl$outlier.rank
})

cols <- lapply(outlierness, function(outl) {
    outl$feature1
})

ranks <- as.data.frame(do.call(cbind, ranks))
colnames(ranks) <- cols
#rownames(ranks) <- names(ranks[[1]])

Compute Spearman correlation and prune the hight correlated ones.

##### Código

In [ ]:
spearman.correlation.acceleration.up <- corr.high(ranks, -1, method="spearman")

In [ ]:
# Remove correlated classifications
remove.high.correlated.ranks <- function(rank.correlations, max.correlation) {
    included.features <- c()
    excluded.features <- c()
    rank.correlations$Var1 <- as.character(rank.correlations$Var1)
    rank.correlations$Var2 <- as.character(rank.correlations$Var2)
    correlated <- rank.correlations[rank.correlations$Freq >= max.correlation,]
    unrelated <- rank.correlations[rank.correlations$Freq < max.correlation,]
    if (nrow(correlated) > 0) {
        for (i in 1:nrow(correlated)) {
            row <- correlated[i,]
    #print(row)
            if (
                !(row[1,]$Var1 %in% included.features) && 
                !(row[1,]$Var1 %in% excluded.features) &&
                !(row[1,]$Var2 %in% included.features)
            ){
                included.features <- c(included.features, row[1,]$Var1)
    #print(paste("include: ", row[1,]$Var1))
    #print(paste("exclude: ", row[1,]$Var2))
                excluded.features <- c(excluded.features, row[1,]$Var2)
            }
        }
    }
    if (nrow(unrelated) > 0) {
        for (i in 1:nrow(unrelated)) {
            row <- unrelated[i,]
            if (
                !(row[1,]$Var1 %in% included.features) &&
                !(row[1,]$Var1 %in% excluded.features)
            ){
                included.features <- c(included.features, row[1,]$Var1)
            }
            if (
                !(row[1,]$Var2 %in% included.features) &&
                !(row[1,]$Var2 %in% excluded.features)
            ){
                included.features <- c(included.features, row[1,]$Var2)
            }
        }
    }
    included.features
}

Consider a threshold of 0.6 among all the selected features.

In [ ]:
CORR.THRESHOLD <- 0.6
#spearman.correlation.acceleration.up <- corr.low(ranks, 0.35, method="spearman")
unrelated.features <- remove.high.correlated.ranks(spearman.correlation.acceleration.up, CORR.THRESHOLD)
while(max(spearman.correlation.acceleration.up$Freq) >= CORR.THRESHOLD) {
    spearman.correlation.acceleration.up <- corr.high(ranks[,colnames(ranks) %in% unrelated.features], -1, method="spearman")
    unrelated.features <- remove.high.correlated.ranks(spearman.correlation.acceleration.up, CORR.THRESHOLD)
    #print(unrelated.features)
}
#corr.high(ranks[,colnames(ranks) %in% unrelated.features], -1, method="spearman")

Selected features:

In [ ]:
unrelated.features
length(unrelated.features)

In [ ]:
save.image(file="anomaliesv3.rdata")

In [ ]:
#load(file="anomaliesv3.rdata")

#### Rank aggregation with Borda Count

In [ ]:
borda.count <- function(ranks) {
    rowSums(ranks)
}

In [ ]:
ranks <- ranks[, unrelated.features]
anoms.ranks <- borda.count(ranks)

In [ ]:
ranks.means <- rowMeans(ranks)
ranks.stdev <- sqrt(rowVars(as.matrix(ranks)))
ranks.relative.stdev <- ranks.stdev / ranks.means

#### Anomalous authors

Aqui apresentamos os top 50 autores anômalos encontrados.

In [ ]:
ranked.authors <- features[order(-anoms.ranks),]
ranked.authors$ranks.relative.stdev <- ranks.relative.stdev[order(-anoms.ranks)]
ranked.authors$rank <- 1:nrow(ranked.authors)
head(ranked.authors[,c("Node", "Node.name", "rank", "academic.birth", "ranks.relative.stdev")], n=110)

In [ ]:
ecdf.data.anomalies <- sapply(anoms.ranks, function(anom) {
        sum(anoms.ranks <= anom) / length(anoms.ranks)
})
ecdf.data.anomalies <- data.frame(score=anoms.ranks, probability=ecdf.data.anomalies)
#ecdf.data.anomalies

In [ ]:
val.top.anoms <- anoms.ranks[order(-anoms.ranks)]
val.top.anoms <- val.top.anoms[1:5]
val.top.anoms

In [ ]:
ecdf.val.top.anoms <- min(ecdf.data.anomalies[ecdf.data.anomalies$score %in% val.top.anoms,]$probability)
score.top.anoms.threshold <- min(val.top.anoms)

In [ ]:
y.labels <- function(x) sprintf("%.2f", x)
p <- plot(ggplot(ecdf.data.anomalies, aes(x=score, y=probability)) +
      geom_line(color = "black", size = 1, linetype="solid") +
      scale_x_continuous(
          #trans="log10", 
          breaks=function(bob) {
              min.val <- min(bob)
              #min.val <- min.val %/% 10 * 10

              max.val <- max(bob)
              step <- (max.val - min.val) / 10
              brks <- seq(min.val, max.val, step)

              pos99 <- sum(brks < score.top.anoms.threshold)

              if (abs(val.top.anoms - brks[pos99]) > 10 &&
                  abs(val.top.anoms - brks[pos99+1]) > 10) {

                  brks <- append(brks, score.top.anoms.threshold, after=pos99)
              } else {
                  if (abs(val.top.anoms - brks[pos99]) <= 10) {
                      brks[pos99] <- score.top.anoms.threshold
                  } else {
                      brks[pos99+1] <- score.top.anoms.threshold
                  }
              }

              if (max.val > 1.5) {
                  ceiling(brks)
              } else {
                  brks <- brks[brks <= 1]
                  brks <- round(brks, 1)
                  if (!1 %in% brks) {
                      brks <- append(brks, 1)
                  }
                  print(brks)
                  brks
              }
          }
      ) +
      scale_y_continuous(
          breaks=sort(c(seq(0, 1, length.out=5), ecdf.val.top.anoms)),
          labels=y.labels(sort(c(seq(0, 1, length.out=5), ecdf.val.top.anoms)))
      ) +
      geom_vline(xintercept=score.top.anoms.threshold, linetype="dotted") + 
      geom_hline(yintercept=ecdf.val.top.anoms, linetype="dotted") + 
      theme(
          # axis.title.x=element_blank(),
          # axis.title.y=element_blank(),
          text = element_text(size=15),
          axis.text.x = element_text(angle = 90, vjust = 0.5, hjust=1)
      ) +
      labs(
        x = "Anomaly score",
        y = "ECDF"
      )
    )

save.plot(p, paste0("anomlay_score_ecdf", ""))

In [ ]:
p <- ggplot(ecdf.data.anomalies, aes(x=score)) +
    geom_histogram() +
    labs(
      x = "Anomaly score",
      y = "Frequency"
    )
m <- mean(ecdf.data.anomalies$score)
sdev <- sd(ecdf.data.anomalies$score)
outliers_count <- sum(ecdf.data.anomalies$score >= (m + 3*sdev))
print(paste("Mean:", m))
print(paste("Sd:", sdev))
print(paste("Outliers:", outliers_count))

idx_mean <- 
    sum(
        ecdf.data.anomalies$score[
            order(ecdf.data.anomalies$score)
        ] <= m
    )

print(paste("Pos mean:", idx_mean))

    sum(
        ecdf.data.anomalies$score[
            order(ecdf.data.anomalies$score)
        ] > m
    )

print(paste("Max: ", max(ecdf.data.anomalies$score)))

p
save.plot(p, paste0("anomlay_score_distribution", ""))

In [ ]:
(m + 3*sdev)

In [ ]:
write.csv(ranked.authors, "/tmp/ranked-authors.csv")

#### Top and middle ranked authors features

In [ ]:
count.n <- 5
feats <- c(
    "rank",
    "academic.birth",
    "all.publications",
    unrelated.features
)

anoms.authors <- ranked.authors[
    1:count.n,
    c(
        "Node", 
        "Node.name", 
        feats
    )]


middle.start <- 2341

normal.authors <- ranked.authors[
    middle.start:(middle.start + count.n-1),
    c(
        "Node", 
        "Node.name", 
        feats
    )]

interested.authors <- rbind(anoms.authors, normal.authors)

interested.authors[,feats] <- round(interested.authors[,feats], 2)

interested.authors$academic.age <- 2021 - interested.authors$academic.birth
interested.authors$publications.day <- 
    (interested.authors$all.publications / (interested.authors$academic.age*261))^-1

In [ ]:
anoms.authors$Node
anoms.authors

In [ ]:
normal.authors$Node
normal.authors

#### DBLP-like plot

In [ ]:
rownames(interested.authors) <- as.character(interested.authors$Node)
x <- lapply(interested.authors$Node[1:count.n], function(author) {
    y <- ggplot(
            all.t[all.t$Node == author,], 
            aes(x=Time, y=diff.papers)
       ) +
       geom_bar(stat="identity", fill="#900c3f") +
       scale_x_discrete(
           breaks=seq(10,20),
       ) +
       theme(
           axis.title.x=element_blank(),
           axis.title.y=element_blank(),
           text = element_text(size=15),
       ) 
    plot(y)
    save.plot(
        y, 
        paste0("simulate.dblp.", interested.authors[as.character(author),"rank"]),
        h = 400
    )
})

In [ ]:
interested.authors$rank

#### Prepare latex tables

###### Features table

In [ ]:
latex.table <- interested.authors
the.column.names.latex <- colnames(latex.table)

get.t <- function(val) {
    year <- str_match(val, ".*(\\d+)\\.years.*")
    #print(year)
    year <- year[1, 2]
    #print(year)
    if (is.na(year)) {
        year <- 1
    }
    year
}

get.var.mean <- function(val) {
    varmean <- ""
    if (grepl("var.", val, fixed=T)) {
        varmean <- "SD"
    } else if (grepl("mean.", val, fixed=T)) {
        varmean <- "MEAN"
    }
    varmean
}

compute.latex.columns <- function(columns.names) {
    
    columns.names <- sapply(columns.names, function (col) {

        feat <- ""
        year <- ""
        varmean <- ""

        if (grepl("diff.cumulative.coauthors", col, fixed=T)) {
            feat <- "RCGS"
            year <- get.t(col)
            varmean <- get.var.mean(col)
        } else if (grepl("diff.papers", col, fixed=T)) {
            feat <- "RAGS"
            year <- get.t(col)
            varmean <- get.var.mean(col)
        } else if (grepl("diff.2.cumulative.coauthors.", col, fixed=T)) {
            feat <- "RCGA"
            year <- get.t(col)
            varmean <- get.var.mean(col)
        } else if (grepl("diff.2.papers.", col, fixed=T)) {
            feat <- "RAGA"
            year <- get.t(col)
            varmean <- get.var.mean(col)
        }

        if (feat == "" && grepl("entropy", col, fixed=T)) {
            feat <- "entropy"
        }

        if (feat == "") {
            feat <- col
        } else if (feat != "entropy") {
            feat <- paste0("T_", year, " ", feat, " ", varmean)
        }
        feat
    })
}

colnames(latex.table) <- compute.latex.columns(the.column.names.latex)

# colnames(latex.table)

latex.table$Node <- NULL
latex.table$Node.name <- NULL
latex.table$academic.birth <- NULL
latex.table$all.publications <- NULL
latex.table$academic.age <- NULL
latex.table$publications.day <- NULL

latex.table$Author <- paste("$A_{", latex.table$rank, "}$", sep="")

latex.table$rank <- NULL

the.column.names.latex <- sort(colnames(latex.table))

# the.column.names.latex

the.column.names.latex <- c(
    "Author",
    the.column.names.latex[
        (the.column.names.latex != "Author")
    ]
)


the.column.names.latex <- c(
    the.column.names.latex[
        (the.column.names.latex != "mean.author.minus.venue.context.zscore")
    ],
    "mean.author.minus.venue.context.zscore"
)

the.column.names.latex <- c(
    the.column.names.latex[
        (the.column.names.latex != "entropy")
    ],
    "entropy"
)

col_names_map <- data.frame(
    col_id=the.column.names.latex, 
    col_name=the.column.names.latex
)
col_names_map$col_id <- as.character(col_names_map$col_id)
col_names_map$col_name <- as.character(col_names_map$col_name)

rownames(col_names_map) <- the.column.names.latex

col_names_map["mean.author.minus.venue.context.zscore","col_name"] <- "Z-SCORE"
col_names_map["entropy",]$col_name <- "1 - E"

for (col in names(latex.table)) {
    if (grepl("SD", col, fixed=T)) {
        latex.table[,col] <- sqrt(latex.table[,col])
    }
}

latex.table <- latex.table[,the.column.names.latex]

names(latex.table) <- col_names_map$col_name

names(latex.table)

latex.table <- latex.table[
    ,
    c(
        "Author", 
        "T_1 RCGS MEAN",
        "T_1 RAGS MEAN",
        "T_2 RCGS SD",
        "T_2 RAGA SD",
        'T_3 RCGA SD',
        "1 - E",
        "Z-SCORE"
    )
]

#latex.table <- latex.table[,c(the.column.names.latex[1:7], "entropy")]
rownames(latex.table) <- NULL

#print(paste(colnames(latex.table), sep = " & "))
#x <- apply(latex.table, 1, function(row) {
#    values <- row[-1]
#    values <- sprintf("%0.2f", as.numeric(values))
#    values <- c(row[1], values)
#    paste(values, sep = " & ")
#})

#latex.table <- t(x)
#colnames(latex.table) <- colnames(latex.table)


print(xtable(latex.table, type = "latex"))

##### Ranks table

In [ ]:
latex.table <- ranks
for(col in colnames(latex.table)) {
    latex.table[,col] <- as.integer(nrow(latex.table) - latex.table[,col] + 1)
}

latex.table$Node <- features$Node
latex.table <- latex.table[order(-anoms.ranks),]
latex.table$Author <- paste("$A_{", 1:nrow(latex.table), "}$", sep="")
latex.table <- latex.table[
    latex.table$Node %in% interested.authors$Node,
]
the.column.names.latex <- colnames(latex.table)
colnames(latex.table) <- compute.latex.columns(the.column.names.latex)
latex.table$Node <- NULL
latex.table$Node.name <- NULL
latex.table$academic.birth <- NULL
latex.table$all.publications <- NULL


latex.table <- latex.table[,col_names_map$col_id]

names(latex.table) <- col_names_map$col_name

latex.table <- latex.table[
    ,
    c(
        "Author", 
        "T_1 RCGS MEAN",
        "T_1 RAGS MEAN",
        "T_2 RCGS SD",
        "T_2 RAGA SD",
        'T_3 RCGA SD',
        "1 - E",
        "Z-SCORE"
    )
]

print(xtable(latex.table, type = "latex"))

In [ ]:
mean.pubs.anoms <- mean(interested.authors[
    interested.authors$Node %in% anoms.authors$Node,
    "publications.day"
])

mean.pubs.normal <- mean(interested.authors[
    interested.authors$Node %in% normal.authors$Node,
    "publications.day"
])

if (F) {
print (
    paste0("Coauthors A ",
        sum(
            features[
                features$Node %in% interested.authors$Node[1:count.n],
            ]$all.coauthors
        )
    )
)

print (
    paste0("Coauthors B ",
        sum(
            features[
                features$Node %in% interested.authors$Node[count.n:nrow(interested.authors)],
            ]$all.coauthors
        )
    )
)
}
    
print (
    paste0("Mean Coauthors A ",
        sum(
            features[
                features$Node %in% interested.authors$Node[1:count.n],
            ]$all.coauthors
        ) / 5
    )
)

print (
    paste0("Mean Coauthors B ",
        sum(
            features[
                features$Node %in% interested.authors$Node[count.n:nrow(interested.authors)],
            ]$all.coauthors
        ) / 5
    )
)

if(F){
print(
    paste0("Publications A ", sum(interested.authors$all.publications[1:count.n]))
)
print(
    paste0("Publications B ", sum(interested.authors$all.publications[count.n:nrow(interested.authors)]))
)
}

print(
    paste0("Mean Publications A ", sum(interested.authors$all.publications[1:count.n]) / 5)
)
print(
    paste0("Mean Publications B ", sum(interested.authors$all.publications[count.n:nrow(interested.authors)]) / 5)
)


print(paste0("Days per paper anomalous: ", mean.pubs.anoms))
print(paste0("Days per paper normal: ", mean.pubs.normal))

In [ ]:
ranks$Node <- features$Node

#### Top and middle-ranked plots

In [ ]:
plot.author.behavior.3 (
    anoms.authors$Node, 
    normal.authors$Node,
    all.t,
    c(names(features.names.1)),
    features.names.1 
)

#### Top 10 anomalous authors' behavior

In [ ]:
plot.authors.behavior(features[order(-anoms.ranks),]$Node[1:10], all.t, "anomalies")